# Intro to Training and Configurations

First we import fibad and create a new fibad object, instantiated (implicitly), with the default configuration file.

In [ ]:
import fibad

fibad_instance = fibad.Fibad()

For this demo, we'll make a few adjustments to the default configuration settings that the `fibad` object was instantiated with. By accessing the `.config` attribute of the fibad instance, we can modify any configuration value. Here we change which built in model to use, the dataset, batch size, number of epochs for training.

In [ ]:
fibad_instance.config["model"]["name"] = "ExampleCNN"
fibad_instance.config["data_set"]["name"] = "CifarDataSet"
fibad_instance.config["data_loader"]["batch_size"] = 64
fibad_instance.config["train"]["epochs"] = 2

We call the `.train()` method to train the model

In [ ]:
fibad_instance.train()

The output of the training will be stored in a time-stamped directory under the `./results/`. By default, a copy of the final configuration used in training is persisted as `runtime_config.toml`. To run fibad again with the same configuration, you can reference the runtime_config.toml file.

If running in another notebook, instantiate a fibad object like so:
```
new_fibad_instance = fibad.Fibad(config_file='./results/<timestamped_directory>/runtime_config.toml')
```

Or from the command line:
```
>> fibad train --runtime-config ./results/<timestamped_directory>/runtime_config.toml
```

Fibad automatically records training metrics so that they can be examined using Tensorboard.

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir ./results

# If you are running on a remote server and want to run tensorboard there;
# you need to pass additional argments for port forwarding to work.
# For example
# %tensorboard --logdir ./results --bind_all --port 8888
# will start tensorboard on port 8888 and you may need to forward that
# port to your local machine using
# ssh -N -L 8888:<name_of_machine>:8888 <username@server.com>

Once a model has been trained, we can use the model weights file to run inference on. Here we update the configuration in the `fibad_instance` object to specify that we want to use a specific model weights file, and that we want our dataset to be 100% test data.

If you are running this locally, you'll need to update the path to your local model weights file.

In [ ]:
# Set this to the path of the example.pth file that was created by the call to fibad_instance.train().
# It should be something like `.../results/<timestamp>-train/example_model.pth`.
fibad_instance.config["infer"]["model_weights_file"] = ""

fibad_instance.config["data_set"]["test_size"] = 1.0
fibad_instance.config["data_set"]["train_size"] = 0.0
fibad_instance.config["data_set"]["validate_size"] = 0.0
fibad_instance.config["data_loader"]["batch_size"] = 128

The following will run inference on the specified dataset.

In [ ]:
# Uncomment the following line after setting the model_weights_file in the previous cell

# fibad_instance.infer()

TODO: Add a cell to plot a confusion matrix.